In [2]:
import sys
sys.path.append('..')

In [11]:
import numpy as np
import pandas as pd
import sklearn
from core.dataset import *
import xgboost as xgb

In [4]:
load_dir = 'D:/Data/hoffmanlab/featureselection/data/'
ligands = ['CpG', 'FLA', 'FSL', 'LPS', 'P3K', 'PIC', 'R84', 'TNF']
sheet_type = 'am'

In [8]:
dataset = Data(load_dir, ligands, sheet_type, merge=True)

In [10]:
dataset.shape
# 14376 entries, 984 features, last column is the corresponding class label

(14376, 985)

In [34]:
X = dataset[:, [i for i in range(984)]]
y = dataset[:, [984]]

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [37]:
model = xgb.XGBClassifier()

In [38]:
model.fit(X_train, y_train)

[17:04:35] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=16,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [52]:
pred = model.predict(X_val)

In [44]:
pred_df = pd.DataFrame(pred)
y_val_df = pd.DataFrame(y_val)

In [54]:
acc = 0
for x, y in zip(pred, y_val):
    if x == y:
        acc += 1

In [56]:
print(f'Default accuracy: {acc / len(y_val)}

0.6467315716272601